# Environment Setup

In [1]:
import glob
import json
import pandas as pd
from datetime import datetime

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, functions as F, types as T, Window as W

In [2]:
# Set up the Spark configuration and context
conf = SparkConf().setAppName("MyApp").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Set up the Spark session
spark = SparkSession.builder \
    .appName("BatchProcessor") \
    .config("spark.driver.extraJavaOptions", "-Xss4m") \
    .config("spark.executor.extraJavaOptions", "-Xss4m") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 23:19:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/06 23:19:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Data Preparation

In [3]:
schema = T.StructType([
    T.StructField("authors", T.ArrayType(T.StringType()), True),
    T.StructField("date_google", T.StringType(), True),
    T.StructField("date_metadata", T.StringType(), True),
    T.StructField("date_published", T.StringType(), True),
    T.StructField("date_target", T.StringType(), True),
    T.StructField("description", T.StringType(), True),
    T.StructField("explanation", T.StringType(), True),
    T.StructField("groq_usage", T.StringType(), True),
    T.StructField("metadata", T.MapType(T.StringType(), T.StringType()), True),
    T.StructField("rating_democrats", T.FloatType(), True),
    T.StructField("rating_republicans", T.FloatType(), True),
    T.StructField("source_url", T.StringType(), True),
    T.StructField("summary", T.StringType(), True),
    T.StructField("text", T.StringType(), True),
    T.StructField("title", T.StringType(), True),
    T.StructField("url", T.StringType(), True)
])
news_df = spark.read.option("multiline", "true").json("../news_ratings/data/", schema=schema)
news_df.show(n=5)

+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|        date_google|       date_metadata|     date_published|        date_target|         description|         explanation|          groq_usage|            metadata|rating_democrats|rating_republicans|          source_url|             summary|                text|               title|                 url|
+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            

In [4]:
market_df = spark.read.csv("../stocks_data/ticker_data.csv", header=True, inferSchema=True)
market_df.show(n=5)

+------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------+
|Ticker|               Date|              Open|              High|               Low|             Close|         Adj Close|     Volume|
+------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------+
|  COIN|2023-11-16 14:30:00| 97.68000030517578| 97.79000091552734| 95.04000091552734| 97.01000213623047| 97.01000213623047|  2528149.0|
|   XOM|2023-11-16 14:30:00| 102.9000015258789| 103.2699966430664|101.98999786376953|102.37999725341797|102.37999725341797|  4463991.0|
| ^GSPC|2023-11-16 14:30:00|    4497.080078125|    4511.990234375|   4495.7001953125|           4505.75|           4505.75|        0.0|
|   SPY|2023-11-16 14:30:00| 449.2200012207031|450.55999755859375| 449.1300048828125|449.95001220703125|449.95001220703125|1.0474895E7|
|   XLI|2023-11-16 14:30:00|105.02999877929688| 

In [5]:
market_df.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: double (nullable = true)



# Data Pre-processing

In [6]:
start_time = datetime(2024, 9, 1)
end_time = datetime(2024, 12, 1)

In [7]:
news_df = news_df.withColumn("published_at", F.coalesce("date_google", "date_metadata", "date_published")) \
    .withColumn("published_at", F.to_timestamp("published_at")) \
    .withColumn("date_target", F.to_date("date_target"))
news_df = news_df.select('published_at', 'date_target', 'rating_democrats', 'rating_republicans', 'title', 'summary', 'url')
news_df = news_df.na.drop(subset=["published_at"])
news_df.sample(fraction=0.01, seed=1947).show(n=5)

+-------------------+-----------+----------------+------------------+--------------------+--------------------+--------------------+
|       published_at|date_target|rating_democrats|rating_republicans|               title|             summary|                 url|
+-------------------+-----------+----------------+------------------+--------------------+--------------------+--------------------+
|2024-02-14 22:16:42| 2024-02-13|             4.0|              -3.0|New York special ...| Democrat Tom Suo...|https://www.thegu...|
|2024-02-15 03:46:44| 2024-02-13|             0.0|               0.0|Prabowo Subianto ...| Prabowo Subianto...|https://www.aljaz...|
|2024-04-29 19:13:08| 2024-02-22|             1.0|              -1.0|US Troops to Stay...| There have been ...|https://arabcente...|
|2024-02-24 04:21:17| 2024-02-23|             0.0|              -2.0|February 23 - 202...| South Carolina s...|https://www.cnn.c...|
|2024-02-26 18:00:00| 2024-02-25|             2.0|              -2.0|

In [ ]:
market_df = (
    market_df
    .filter(F.col("Date") > start_time)
    .filter(F.col("Date") < end_time)
    .filter(F.col("Ticker").isin(["JPM", "GS"]))
    .withColumn("hour_bucket", F.date_trunc("hour", F.col("Date")))
    .repartition("Ticker")
)
news_df = (
    news_df
    .filter(F.col("published_at") > start_time)
    .filter(F.col("published_at") < end_time)
    .withColumn("hour_bucket", F.date_trunc("hour", F.col("published_at")))
)
hours_df = (
    spark.createDataFrame([(start_time, end_time)], ["start_time", "end_time"])
    .select(F.explode(F.sequence(F.col("start_time"), F.col("end_time"), F.expr("INTERVAL 1 HOUR"))).alias("hour_bucket"))
)
tickers_df = market_df.select("Ticker").distinct()

In [9]:
market_df = hours_df.crossJoin(tickers_df).join(market_df, on=["hour_bucket", "Ticker"], how="left")
news_df = hours_df.join(news_df, on="hour_bucket", how="left")

# Feature Engineering

In [ ]:
time_windows = [
    # 6,      # 6 hours
    # 12,     # 12 hours
    24,     # 1 day
    7*24,   # 1 week
    14*24,  # 2 weeks
    # 28*24,  # 4 weeks
]
# --------------------
statistics = [
    "count",
    "mean",
    "std",
    "min",
    "max",
    "spread",
]
# --------------------
ticker_cols = [
    "Open",
    "High",
    "Low",
    "Close",
    "Adj Close",
    "Volume"
]
# --------------------
news_cols = [
    "rating_republicans",
    "rating_democrats",
]

In [12]:
def calculate_rolling_stats(df, cols, partition_col=None):
    stat_exprs = []
    if partition_col:   stat_exprs.append(F.col(partition_col))
    for window_hours in time_windows:
        window_spec = (
            W
            .partitionBy(partition_col if partition_col else [])
            .orderBy(F.col("hour_bucket").cast("timestamp").cast("long"))
            .rangeBetween(-1 * window_hours * 3600, 0)
        )
        for col in cols:
            for stat in statistics:
                field_name = f"rolling_{window_hours}h_{col}_{stat}"
                if   stat == "count":   stat_exprs.append(F.count(col).over(window_spec).alias(field_name))
                elif stat == "mean":    stat_exprs.append(F.mean(col).over(window_spec).alias(field_name))
                elif stat == "std":     stat_exprs.append(F.stddev(col).over(window_spec).alias(field_name))
                elif stat == "min":     stat_exprs.append(F.min(col).over(window_spec).alias(field_name))
                elif stat == "max":     stat_exprs.append(F.max(col).over(window_spec).alias(field_name))
                elif stat == "median":  stat_exprs.append(F.approx_percentile(col, 0.5, 10).over(window_spec).alias(field_name))
                elif stat == "spread":  stat_exprs.append((F.max(col).over(window_spec) - F.min(col).over(window_spec)).alias(field_name))
                else: raise ValueError(f"Unknown statistic: {stat}")
    
    # Compute all statistics for the current column and window
    result_df = df.select("hour_bucket", *stat_exprs).distinct()

    return result_df

In [13]:
# %%script false --no-raise-error
market_rollstats_df = calculate_rolling_stats(market_df, ticker_cols, "Ticker").cache()
news_rollstats_df = calculate_rolling_stats(news_df, news_cols).cache()

24/12/06 23:20:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/06 23:20:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/06 23:20:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
news_rollstats_df.show(n=20)

24/12/06 23:20:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/06 23:20:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/06 23:20:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/06 23:20:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


2185
+-------------------+-----------------------------------+----------------------------------+---------------------------------+---------------------------------+---------------------------------+------------------------------------+---------------------------------+--------------------------------+-------------------------------+-------------------------------+-------------------------------+----------------------------------+------------------------------------+-----------------------------------+----------------------------------+----------------------------------+----------------------------------+-------------------------------------+----------------------------------+---------------------------------+--------------------------------+--------------------------------+--------------------------------+-----------------------------------+------------------------------------+-----------------------------------+----------------------------------+----------------------------------+---

In [ ]:
market_rollstats_df.show(n=20)

4370


+-------------------+------+---------------------+--------------------+-------------------+-------------------+-------------------+----------------------+---------------------+--------------------+-------------------+-------------------+-------------------+----------------------+--------------------+-------------------+-------------------+------------------+------------------+---------------------+----------------------+---------------------+--------------------+--------------------+--------------------+-----------------------+--------------------------+-------------------------+------------------------+------------------------+------------------------+---------------------------+-----------------------+----------------------+---------------------+---------------------+---------------------+------------------------+----------------------+---------------------+--------------------+--------------------+--------------------+-----------------------+----------------------+------------------

In [ ]:
# %%script false --no-raise-error
sc.stop()       # Stop the Spark contex